In [41]:
import os
import cv2
import pandas as pd
import mediapipe as mp


In [42]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)


In [43]:
import numpy as np

def get_landmarks(image_path):
    try:
        with open(image_path, "rb") as f:
            file_bytes = np.asarray(bytearray(f.read()), dtype=np.uint8)
            image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    except Exception as e:
        print(f"Ошибка чтения файла: {image_path}\n{e}")
        return None

    if image is None:
        print(f"Не удалось загрузить изображение: {image_path}")
        return None

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_rgb)
    if not results.multi_face_landmarks:
        print(f"Лицо не найдено: {image_path}")
        return None

    return results.multi_face_landmarks[0]


In [44]:
def measure_features(landmarks):
    def dist(p1, p2):
        return ((p1.x - p2.x)**2 + (p1.y - p2.y)**2)**0.5

    # Координаты нужных точек
    lm = landmarks.landmark

    try:
        # Признак 1: Лоб вертикальный (дистанция между висками / высота от подбородка до лба)
        forehead_width = dist(lm[127], lm[356])
        face_height = dist(lm[152], lm[10])
        vertical_forehead_ratio = forehead_width / face_height

        # Признак 2: Высота бровей (расстояние от центра глаза до брови)
        brow_height = dist(lm[159], lm[386])

        # Признак 3: Форма головы (высота / ширина головы)
        head_height = dist(lm[10], lm[152])
        head_width = dist(lm[234], lm[454])
        head_ratio = head_height / head_width

        return {
            "forehead_ratio": vertical_forehead_ratio,
            "brow_height": brow_height,
            "head_ratio": head_ratio,
        }

    except Exception as e:
        print(f"Ошибка при измерении признаков: {e}")
        return None


In [45]:
def build_dataframe(base_path):
    rational_data, irrational_data = [], []

    for folder in sorted(os.listdir(base_path)):
        folder_path = os.path.join(base_path, folder)
        if not os.path.isdir(folder_path):
            continue

        label = 0 if folder == "J" else 1  # J = Рациональные, P = Иррациональные

        for img_name in sorted(os.listdir(folder_path)):
            img_path = os.path.join(folder_path, img_name)
            landmarks = get_landmarks(img_path)
            if not landmarks:
                continue

            features = measure_features(landmarks)
            if not features:
                continue

            if label == 0:
                rational_data.append(features)
            else:
                irrational_data.append(features)

    return pd.DataFrame(rational_data), pd.DataFrame(irrational_data)


In [46]:
base_path = r"E:\Project VS\Machine-Learning\binary_data"
rational_df, irrational_df = build_dataframe(base_path)

print("Рациональных строк:", len(rational_df))
print("Иррациональных строк:", len(irrational_df))


Лицо не найдено: E:\Project VS\Machine-Learning\binary_data\J\Анастасия Семеренко (12).jpg
Лицо не найдено: E:\Project VS\Machine-Learning\binary_data\J\Анастасия Семеренко (17).jpg
Лицо не найдено: E:\Project VS\Machine-Learning\binary_data\J\Анастасия Яночкина (15).jpg
Лицо не найдено: E:\Project VS\Machine-Learning\binary_data\J\Анастасия Яночкина (17).jpg
Лицо не найдено: E:\Project VS\Machine-Learning\binary_data\J\Анастасия Яночкина (4).jpg
Лицо не найдено: E:\Project VS\Machine-Learning\binary_data\J\Анна Ельникова (11).jpg
Лицо не найдено: E:\Project VS\Machine-Learning\binary_data\J\Анна Ельникова (12).jpg
Лицо не найдено: E:\Project VS\Machine-Learning\binary_data\J\Анна Задорина (17).jpg
Лицо не найдено: E:\Project VS\Machine-Learning\binary_data\J\Артем Реасов (1).jpg
Лицо не найдено: E:\Project VS\Machine-Learning\binary_data\J\Беренгард Дария — копия.jpg
Лицо не найдено: E:\Project VS\Machine-Learning\binary_data\J\Варвара Короткевич (4).jpg
Лицо не найдено: E:\Project VS

In [47]:
columns = rational_df.columns.intersection(irrational_df.columns)

for col in columns:
    r_mean = rational_df[col].mean()
    r_std = rational_df[col].std()

    i_mean = irrational_df[col].mean()
    i_std = irrational_df[col].std()

    print(f"{col}:")
    print(f"  Рациональные:")
    print(f"    Среднее значение      = {r_mean:.4f}")
    print(f"    Стандартное отклонение = {r_std:.4f}")
    print(f"  Иррациональные:")
    print(f"    Среднее значение      = {i_mean:.4f}")
    print(f"    Стандартное отклонение = {i_std:.4f}")
    print("-" * 50)


forehead_ratio:
  Рациональные:
    Среднее значение      = 0.9853
    Стандартное отклонение = 0.1953
  Иррациональные:
    Среднее значение      = 0.9428
    Стандартное отклонение = 0.1861
--------------------------------------------------
brow_height:
  Рациональные:
    Среднее значение      = 0.3116
    Стандартное отклонение = 0.0638
  Иррациональные:
    Среднее значение      = 0.2976
    Стандартное отклонение = 0.0605
--------------------------------------------------
head_ratio:
  Рациональные:
    Среднее значение      = 1.0708
    Стандартное отклонение = 0.2172
  Иррациональные:
    Среднее значение      = 1.1169
    Стандартное отклонение = 0.2171
--------------------------------------------------
